<a href="https://colab.research.google.com/github/bognev/transfer_learning_demo/blob/main/georges_transfer_learning_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [173]:
from PIL import Image
import requests
import csv
from random import shuffle
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

In [174]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [175]:
%matplotlib inline
plt.ion()

In [176]:
def download_images(url_list, dataset_folder):
    for url in tqdm(url_list):
        img = Image.open(requests.get(url, stream=True).raw)
        filename = url.split("/")[-1]
        img.save(dataset_folder + filename, "JPEG")
    print("done")

def load_dataset(file, dataset_folder_train, dataset_folder_val):
    url_list = []
    with open(file, 'r') as fd:
        reader = csv.reader(fd)
        for url in reader:
            url_list.append(url[0])
        shuffle(url_list)
        length = len(url_list)
        download_images([url_list[i] for i in range(0, int(0.8 * length))], dataset_folder_train)
        download_images([url_list[i] for i in range(int(0.8 * length), length)], dataset_folder_val)

def full_load():
    path = F"/content/gdrive/My Drive" 
    georges = path + "/georges.csv"
    dataset_folder_train = path + "/georges_data/train/georges/"
    dataset_folder_val = path + "/georges_data/val/georges/"
    load_dataset(georges, dataset_folder_train, dataset_folder_val)

    non_georges = path + "/non_georges.csv"
    dataset_folder_train = path + "/georges_data/train/non_georges/"
    dataset_folder_val = path + "/georges_data/val/non_georges/"
    load_dataset(non_georges, dataset_folder_train, dataset_folder_val)




In [177]:
# full_load() #uncomment for downloading images into google drive

In [178]:
def train_model(model, criterion, optimizer, scheduler, dataloaders, device, dataset_sizes, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [179]:
def confusion_matrix_model(model, dataloaders, device, class_names, num_images=100):
    was_training = model.training
    model.eval()
    images_so_far = 0    
    confusion_matrix = torch.zeros(2, 2)
    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['val']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            for t, p in zip(labels.view(-1), preds.view(-1)):
                confusion_matrix[t.long(), p.long()] += 1
        model.train(mode=was_training)
        
    print("Confusion matrix:")    
    print(confusion_matrix.numpy())
    pc = confusion_matrix.diag()/confusion_matrix.sum(1)
    print("Per-class accuracy:")
    print(class_names[0] + ": " + str(pc[0].numpy()))
    print(class_names[1] + ": " + str(pc[1].numpy()))

In [180]:
def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated

In [181]:
######################################################################
# Visualizing the model predictions
def visualize_model(model, dataloaders, device, class_names, num_images=6):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['val']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                ax.set_title('predicted: {}'.format(class_names[preds[j]]))
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)

In [182]:
# Data augmentation and normalization for training
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.RandomCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = F"/content/gdrive/My Drive/georges_data/"
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                            data_transforms[x])
                    for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=16,
                                                shuffle=True, num_workers=4)
                for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)


cuda:0


In [ ]:
######################################################################
# Visualize images
inputs, classes = next(iter(dataloaders['train']))
out = torchvision.utils.make_grid(inputs[:4])
imshow(out, title=[class_names[x] for x in classes[:4]])
inputs, classes = next(iter(dataloaders['train']))
out = torchvision.utils.make_grid(inputs[:4])
imshow(out, title=[class_names[x] for x in classes[:4]])
inputs, classes = next(iter(dataloaders['train']))
out = torchvision.utils.make_grid(inputs[:4])
imshow(out, title=[class_names[x] for x in classes[:4]])
inputs, classes = next(iter(dataloaders['train']))
out = torchvision.utils.make_grid(inputs[:4])
imshow(out, title=[class_names[x] for x in classes[:4]])

In [184]:
######################################################################
# Finetuning the convnet
model_ft = models.resnet18(pretrained=True)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 2)
model_ft = model_ft.to(device)
criterion = nn.CrossEntropyLoss()

optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)
######################################################################
# Train and evaluate

model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler, dataloaders, device, dataset_sizes,
                        num_epochs=20)

######################################################################

Epoch 0/19
----------
train Loss: 0.4889 Acc: 0.7627
val Loss: 0.3119 Acc: 0.8700

Epoch 1/19
----------
train Loss: 0.3847 Acc: 0.8311
val Loss: 0.2663 Acc: 0.8968

Epoch 2/19
----------
train Loss: 0.3292 Acc: 0.8581
val Loss: 0.3167 Acc: 0.8708

Epoch 3/19
----------
train Loss: 0.3242 Acc: 0.8648
val Loss: 0.2399 Acc: 0.9102

Epoch 4/19
----------
train Loss: 0.2829 Acc: 0.8800
val Loss: 0.2123 Acc: 0.9220

Epoch 5/19
----------
train Loss: 0.2888 Acc: 0.8761
val Loss: 0.2223 Acc: 0.9220

Epoch 6/19
----------
train Loss: 0.2475 Acc: 0.8985
val Loss: 0.2735 Acc: 0.8977

Epoch 7/19
----------
train Loss: 0.2225 Acc: 0.9042
val Loss: 0.2148 Acc: 0.9161

Epoch 8/19
----------
train Loss: 0.2008 Acc: 0.9207
val Loss: 0.1985 Acc: 0.9287

Epoch 9/19
----------
train Loss: 0.1976 Acc: 0.9187
val Loss: 0.2084 Acc: 0.9245

Epoch 10/19
----------
train Loss: 0.1963 Acc: 0.9159
val Loss: 0.1968 Acc: 0.9362

Epoch 11/19
----------
train Loss: 0.1896 Acc: 0.9224
val Loss: 0.1820 Acc: 0.9362

Ep

In [185]:
confusion_matrix_model(model_ft, dataloaders, device, class_names, num_images=100)

Confusion matrix:
[[453.  66.]
 [ 27. 646.]]
Per-class accuracy:
georges: 0.87283236
non_georges: 0.9598811


In [ ]:

visualize_model(model_ft, dataloaders, device, class_names, num_images=2)
visualize_model(model_ft, dataloaders, device, class_names, num_images=2)
visualize_model(model_ft, dataloaders, device, class_names, num_images=2)
visualize_model(model_ft, dataloaders, device, class_names, num_images=2)
plt.ioff()
plt.show()